# Cut Model
My name for a non-dense model that is just a dense model with set of connections that are *cut* from layer to layer

## Imports

In [ ]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

## CutModelBuilder

In [ ]:
class CutModelBuilder(Model):
    def __init__(self, input_shape):
        super(CutModelBuilder,Self).__init__()
        self.input_layer = Input(shape=input_dims)
        self.next_layer = input_layer
        
    def build(self, output_dims):
        self.next_layer = Dense(output_dims)(self.input_layer)
        return self
    
    def _cutOnce(self, cut_node):
        layer1 = Lambda(lambda x: x[:,:,0:cut_node])
        layer2 = Lambda(lambda x: x[:,:,0:])
        
    def cut(self, cut_nodes):
        layer = Lambda(lambda x: x[:,:,0:64])